In [1]:
import numpy as np

### Arguments

In [2]:
# 01 to allow for up to 99 runs

run_no = '01'

### Functions to Read Files and Extract Energies

In [3]:
def getEnergyMBAR(input_file):
    '''
    Finds MBAR free energy from .dat input file.
    '''
    with open(input_file,'r') as file:
        read_file = file.readlines()
    
    energy =''
    standard_dev =''

    for i, line in enumerate(read_file):
        if line.startswith('#MBAR free energy difference in kcal/mol:'):
            energy=read_file[i+1].split()[0][:-1]
            standard_dev=read_file[i+1].split()[1][:-1]
        
    return (energy, standard_dev)

In [4]:
def getEnergyTI(input_file):
    '''
    Finds TI free energy from .dat input file.
    '''
    with open(input_file,'r') as file:
        read_file = file.readlines()
    
    energy =''

    for i, line in enumerate(read_file):
        if line.startswith('#TI free energy difference in kcal/mol:'):
            energy=read_file[i+1].split()[0][:-1]
        
    return energy

In [5]:
def getLJcorr(input_file):
    '''Finds LJ correction from .dat file'''
    
    with open(input_file,'r') as file:
        read_file = file.readlines()
    
    correction = read_file[0].split()[2]
    standard_dev= read_file[0].split()[4]
        
    return (correction,standard_dev)

In [6]:
def getStdState(input_file):
    '''Finds standard state correction from .dat file and returns correction as string'''
    
    with open(input_file,'r') as file:
        read_file = file.readlines()
        
    energy=''
    
    for i, line in enumerate(read_file):
        if line.startswith('Free energy change upon removing the restraint and applying standard state conditions'):
            energy=read_file[i].split()[-2]
            
    return energy

### Paths to Files

In [7]:
energy_paths = {'free_discharge':f'free/run0{run_no}/discharge/output/freenrg-MBAR-p-83-overlap.dat',
         'free_vanish':f'free/run0{run_no}/vanish/output/freenrg-MBAR-p-83-overlap.dat',
         'bound_discharge':f'bound/run0{run_no}/discharge/output/freenrg-MBAR-p-83-overlap.dat',
         'bound_vanish':f'bound/run0{run_no}/vanish/output/freenrg-MBAR-p-83-overlap.dat'}

### Get Energies

In [8]:
energies_mbar = {}

for section in energy_paths:
    energies_mbar[section]=float(getEnergyMBAR(energy_paths[section])[0])

In [9]:
sd_mbar = {}

for section in energy_paths:
    sd_mbar[section]=float(getEnergyMBAR(energy_paths[section])[1])

In [10]:
energies_ti = {}

for section in energy_paths:
    energies_ti[section]=float(getEnergyTI(energy_paths[section]))

### Get Corrections

In [11]:
free_lj_corr_str, free_sd_lj_corr_str = getLJcorr(f'free/run0{run_no}/vanish/output/freenrg-LJCOR.dat')[0:2]
free_lj_corr = float(free_lj_corr_str)
free_sd_lj_corr = float(free_sd_lj_corr_str)

In [12]:
bound_lj_corr_str, bound_sd_lj_corr_str = getLJcorr(f'bound/run0{run_no}/vanish/output/freenrg-LJCOR.dat')[0:2]
bound_lj_corr = float(bound_lj_corr_str)
bound_sd_lj_corr = float(bound_sd_lj_corr_str)

In [13]:
std_state_corr = float(getStdState(f'bound/run0{run_no}/vanish/output/standard-state-s-1-b-4-d-0.25-o-60.dat'))

### Calculate Output

In [14]:
dg_bind_mbar=energies_mbar['free_discharge']+energies_mbar['free_vanish']+free_lj_corr-energies_mbar['bound_discharge']-energies_mbar['bound_vanish']-bound_lj_corr-std_state_corr

In [15]:
dg_bind_ti=energies_ti['free_discharge']+energies_ti['free_vanish']+free_lj_corr-energies_ti['bound_discharge']-energies_ti['bound_vanish']-bound_lj_corr-std_state_corr

In [16]:
sd_dg_bind_mbar=np.sqrt(sd_mbar['bound_discharge']**2+sd_mbar['bound_vanish']**2+sd_mbar['free_discharge']**2+sd_mbar['free_vanish']**2+free_sd_lj_corr**2+bound_sd_lj_corr**2)

In [17]:
var = 0

for stage in sd_mbar:
    var+= sd_mbar[stage]**2
    
var+= free_sd_lj_corr**2
var+= bound_sd_lj_corr**2

sd_dg_bind_mbar = np.sqrt(var)

In [19]:
dg_elec_mbar=energies_mbar['free_discharge']-energies_mbar['bound_discharge']

sd_dg_elec_mbar=np.sqrt(sd_mbar['bound_discharge']**2+sd_mbar['free_discharge']**2)

dg_elec_ti=energies_ti['free_discharge']-energies_ti['bound_discharge']

dg_lj_mbar=energies_mbar['free_vanish']+free_lj_corr-energies_mbar['bound_vanish']-bound_lj_corr

sd_dg_lj_mbar=np.sqrt(sd_mbar['bound_vanish']**2+sd_mbar['free_vanish']**2+free_sd_lj_corr**2+bound_sd_lj_corr**2)

dg_lj_ti=energies_ti['free_vanish']+free_lj_corr-energies_ti['bound_vanish']-bound_lj_corr

### Write Output

In [31]:
for section in energy_paths:
    if section == 'bound_vanish':
        print('Section: ',section)
        print('MBAR estimate:', energies_mbar[section],'+/-',sd_mbar[section],'kcal/mol')
        print('TI estimate:', energies_ti[section],'kcal/mol')
        print('LJ correction: ', bound_lj_corr,'+/-',bound_sd_lj_corr,'kcal/mol')
        print('Standard state correction ', std_state_corr, 'kcal/mol','\n')
        
    elif section == 'free_vanish':
        print('Section: ',section)
        print('MBAR estimate:', energies_mbar[section],'+/-',sd_mbar[section],'kcal/mol')
        print('TI estimate:', energies_ti[section],'kcal/mol')
        print('LJ correction: ', free_lj_corr,'+/-',free_sd_lj_corr,'kcal/mol','\n')

    else:
        print('Section: ',section)
        print('MBAR estimate:', energies_mbar[section],'+/-',sd_mbar[section],'kcal/mol')
        print('TI estimate:', energies_ti[section],'kcal/mol','\n')
    
    
print('###########################################################################')

print('dg_electrostatic from MBAR: ',dg_elec_mbar,' +/-', sd_dg_elec_mbar,'kcal/mol')
print('dg_electrostatic from TI: ',dg_elec_ti,'kcal/mol')    

print('dg_LJ from MBAR: ',dg_lj_mbar,' +/-', sd_dg_elec_mbar,'kcal/mol')
print('dg_LJ from TI: ',dg_lj_ti,'kcal/mol','\n')  
    
print('###########################################################################')

print('dg_bind from MBAR: ',dg_bind_mbar,' +/-', sd_dg_bind_mbar,'kcal/mol')
print('dg_bind from TI: ',dg_bind_ti,'kcal/mol','\n')

print('###########################################################################\n')

Section:  free_discharge
MBAR estimate: 9.006035 +/- 0.01048 kcal/mol
TI estimate: 9.0304 kcal/mol 

Section:  free_vanish
MBAR estimate: -12.148559 +/- 0.04437 kcal/mol
TI estimate: -11.78419 kcal/mol
LJ correction:  0.28897 +/- 0.0157 kcal/mol 

Section:  bound_discharge
MBAR estimate: 11.475786 +/- 0.01015 kcal/mol
TI estimate: 11.21436 kcal/mol 

Section:  bound_vanish
MBAR estimate: -3.161217 +/- 0.0359 kcal/mol
TI estimate: -2.46835 kcal/mol
LJ correction:  0.33601 +/- 0.07128 kcal/mol
Standard state correction  -4.13 kcal/mol 

###########################################################################
dg_electrostatic from MBAR:  -2.4697509999999987  +/- 0.014589479085971507 kcal/mol
dg_electrostatic from TI:  -2.183959999999999 kcal/mol
dg_LJ from MBAR:  -9.034381999999999  +/- 0.014589479085971507 kcal/mol
dg_LJ from TI:  -9.362879999999999 kcal/mol 

###########################################################################
dg_bind from MBAR:  -7.374132999999998  +/- 0.0937